Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [530]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Folder dir for saving and loading files

In [531]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
# model_name = 'mtm_spring'
# model_name = 'mtm_2spring'
model_name = 'mtm_2spring_tendon'
# model_name = 'mtm_4links_parallel'
# model_name = 'mtm_3links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [532]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []
tendon_couplings = []


if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print(model_name)
elif model_name == 'mtm_2spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s, -q4]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print(model_name)
elif model_name == 'mtm_2spring_tendon':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s, -q4]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    tendon_couplings = [(4, 3, 2.0/3.0)] # tau_3 += 2/3*tau_4
    print(model_name)
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.2794, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.2794, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous', 'offset']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']              
elif (model_name == 'test_psm_long2'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*17
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     tendon_couplings=tendon_couplings,
                     dh_convention='mdh',
                     friction_type=friction_type)

mtm_2spring_tendon
['Coulomb', 'viscous', 'offset']


In [535]:
robot_def.dq_for_frame

In [536]:
robot_def.coordinates_joint_type

['R', 'R', 'R', 'R', 'R', 'R', 'R']

In [534]:
robot_def.bary_params

## Create kinematics chain

In [298]:
%time geom = Geometry(robot_def)

Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
CPU times: user 19.8 s, sys: 273 ms, total: 20.1 s
Wall time: 19.9 s


In [348]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [300]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/6
Calculating the link kinetic energy of 2/6
Calculating the link kinetic energy of 3/6
Calculating the link kinetic energy of 4/6
Calculating the link kinetic energy of 5/6
Calculating joint torques...
tau of q1
tau of q2
tau of q4
Adding frictions and springs...
Add tendon coupling torque and motor inertia...
Calculating gregressor...
('input_vars', (q1, q2, q4, dq1, dq2, dq4, ddq1, ddq2, ddq4))
Calculating M, C and G...
Calculating base parameter...
[32 58 59 33 14 27 28 15 30 37 17 29 16 45 46 31 18 11 50 24 13 38 12 36
 25 20 23 10 49 19 40 53 26 54 44 55 51  5 66 43 35 52 42 57 56 41 34 61
 62 67 60 64 39 63 65  8  4  9  1  2 22 47 48  7  0  3 21  6]
base parameter number: 29
Creating H_b function...
Finished creating robot dynamics
CPU times: user 7.12 s, sys: 518 ms, total: 7.63 s
Wall time: 6.42 s


In [301]:
robot_def.bary_params

In [302]:
sympy.Matrix(dyn.base_param)

⎡                -0.35842294⋅l2x + 1.0⋅l3x - 0.1⋅m₃                ⎤
⎢                                                                  ⎥
⎢                   1.0⋅l2x + 1.0⋅l5x + 0.279⋅m₃                   ⎥
⎢                                                                  ⎥
⎢                             1.0⋅l5y                              ⎥
⎢                                                                  ⎥
⎢                             1.0⋅l3y                              ⎥
⎢                                                                  ⎥
⎢                       1.0⋅L2xy + 1.0⋅L5xy                        ⎥
⎢                                                                  ⎥
⎢                       1.0⋅L3xy + 1.0⋅L4xy                        ⎥
⎢                                                                  ⎥
⎢                  1.0⋅L3xz + 1.0⋅L4xz - 0.1⋅l5z                   ⎥
⎢                                                                  ⎥
⎢                 1.0⋅L2xz + 1.0⋅L

In [303]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

Saved 29 parameters


In [304]:
for tendon_coupling in dyn.rbt_def.tendon_couplings:
    src_frame, dst_frame, k = tendon_coupling
    dq_src = dyn.rbt_def.dq_for_frame[src_frame]
    dq_dst = dyn.rbt_def.dq_for_frame[dst_frame]
    src_index = dyn.rbt_def.d_coordinates.index(dq_src)

    for a in range(len(dyn.rbt_def.d_coordinates)):
        dq_da = sympy.diff(dq_dst, dyn.rbt_def.d_coordinates[a])
        #tau_c[a] += dq_da * k * tau_csf[src_index]
        print(a, dq_da, k)
dyn.rbt_def.tendon_couplings 
2/3

---
# 3 - Excitation trajectory optimization

In [308]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
#model_name = 'mtm_spring'
# model_name = 'mtm_2spring'
#model_name = 'mtm_2spring_tendon'
model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [309]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [311]:
trajectory_name = 'one'
#trajectory_name = 'two'
# trajectory_name = 'two_normalized'

In [313]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm' or model_name == 'mtm_spring'\
    or model_name == 'mtm_2spring' or model_name == 'mtm_2spring_tendon':
    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-40), deg2rad(195), -6*np.pi, 6*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -6*np.pi, 6*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -6*np.pi, 6*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 4
cartesian constraint number: 0
constraint number: 16
frames_constrained: []
cond: 48.8533872827
cond: 48.8533872827
cond: 48.8533860736
cond: 48.8533876507
cond: 48.8533854989
cond: 48.8533856603
cond: 48.8533790809
cond: 48.853387368
cond: 48.8533915692
cond: 48.8533908408
cond: 48.8533906151
cond: 48.8533906264
cond: 48.853389237
cond: 48.8533867685
cond: 48.853384999
cond: 48.8533837505
cond: 48.8533857184
cond: 48.8533848128
cond: 48.8533855232
cond: 48.8533872637
cond: 48.8533869874
cond: 48.8533925703
cond: 48.8533912061
cond: 48.8533905629
cond: 48.8533900603
cond: 48.8533901142
cond: 48.8533900494
cond: 48.8533892443
cond: 48.8533878135
cond: 48.8533869328
cond: 48.8533873439
cond: 48.8533871798
cond: 48.8533897372
cond: 48.8533882492
cond: 48.8533832283
cond: 48.853388035
cond: 48.8533859168
cond: 48.8533851798
cond: 48.8533855958
cond: 48.8533863215
cond: 482.284959635
cond: 37.0665965615
cond: 37.0665965615
cond: 37.0665961205
cond: 37.066597841
co

cond: 221.265285839
cond: 221.265331696
cond: 221.265330788
cond: 221.265313031
cond: 221.265310391
cond: 221.265338351
cond: 221.26530875
cond: 221.265301034
cond: 221.265349353
cond: 221.265340187
cond: 221.265250501
cond: 221.265336509
cond: 221.264854019
cond: 221.26530187
cond: 221.265336446
cond: 221.265295855
cond: 221.265356981
cond: 221.265246565
cond: 221.265380119
cond: 221.26538897
cond: 221.265241636
cond: 221.265266004
cond: 221.265276058
cond: 221.26529863
cond: 221.265375899
cond: 221.265688224
cond: 221.265200196
cond: 221.265309604
cond: 221.265373765
cond: 221.265304133
cond: 221.265398677
cond: 221.265419295
cond: 221.265285902
cond: 221.265368123
cond: 221.26530603
cond: 221.2653111
cond: 221.265317056
cond: 215.426588275
cond: 215.426588275
cond: 215.426596552
cond: 215.426567523
cond: 215.42658994
cond: 215.426594895
cond: 215.426595366
cond: 215.426581759
cond: 215.426608829
cond: 215.426587507
cond: 215.426563296
cond: 215.42662945
cond: 215.426607481
cond: 215

cond: 393.002773117
cond: 109.848368575
cond: 109.848368575
cond: 109.848366228
cond: 109.848374625
cond: 109.848371158
cond: 109.848355702
cond: 109.848355168
cond: 109.848384798
cond: 109.84837804
cond: 109.848355322
cond: 109.848358784
cond: 109.848375394
cond: 109.848385785
cond: 109.848343095
cond: 109.848385884
cond: 109.848133261
cond: 109.848391343
cond: 109.848410936
cond: 109.848367378
cond: 109.84835303
cond: 109.848367982
cond: 109.848449849
cond: 109.848342151
cond: 109.848325686
cond: 109.848374861
cond: 109.848377849
cond: 109.84840604
cond: 109.848352249
cond: 109.84854751
cond: 109.848350308
cond: 109.848348273
cond: 109.848380515
cond: 109.848392103
cond: 109.848367355
cond: 109.848300246
cond: 109.848393353
cond: 109.848409463
cond: 109.848338822
cond: 109.848382286
cond: 109.848334053
cond: 309.217176501
cond: 141.598828298
cond: 141.598828298
cond: 141.598827032
cond: 141.59882826
cond: 141.598836096
cond: 141.598811988
cond: 141.598819126
cond: 141.598848336
cond:

cond: 200.801595754
cond: 200.801660265
cond: 200.801495855
cond: 200.8014379
cond: 200.801502133
cond: 200.801519586
cond: 200.801606178
cond: 200.801491735
cond: 200.801602673
cond: 200.801466273
cond: 200.801602283
cond: 200.801559528
cond: 200.801728081
cond: 200.801511669
cond: 200.801469432
cond: 200.801526961
cond: 200.801668582
cond: 200.801578066
cond: 200.801422003
cond: 200.801573913
cond: 200.801537898
cond: 200.801474482
cond: 200.801626851
cond: 200.801513346
cond: 197.926803763
cond: 197.926803763
cond: 197.926805821
cond: 197.926801018
cond: 197.926810785
cond: 197.926791611
cond: 197.926792749
cond: 197.926810051
cond: 197.926798802
cond: 197.926782493
cond: 197.926805115
cond: 197.9268368
cond: 197.926800825
cond: 197.926759681
cond: 197.926803701
cond: 197.926542452
cond: 197.926890503
cond: 197.926917975
cond: 197.926751593
cond: 197.926718376
cond: 197.926766397
cond: 197.926772877
cond: 197.926865643
cond: 197.926756124
cond: 197.926854827
cond: 197.92674559
cond:

cond: 194.952650183
cond: 194.952653492
cond: 194.952458285
cond: 194.95247895
cond: 194.952526489
cond: 194.952620192
cond: 194.952748198
cond: 194.952723213
cond: 194.952535113
cond: 194.952637037
cond: 194.952653894
cond: 194.952647569
cond: 194.952662598
cond: 194.952621509
cond: 190.031505533
cond: 190.031505533
cond: 190.031515002
cond: 190.031504358
cond: 190.031510981
cond: 190.031501669
cond: 190.031506484
cond: 190.031510727
cond: 190.031498891
cond: 190.031492754
cond: 190.031519753
cond: 190.03154323
cond: 190.031512365
cond: 190.031456699
cond: 190.031503388
cond: 190.031366783
cond: 190.031578477
cond: 190.031599819
cond: 190.031448195
cond: 190.031378452
cond: 190.031432875
cond: 190.031507801
cond: 190.031555654
cond: 190.031474432
cond: 190.031540522
cond: 190.031438314
cond: 190.031525737
cond: 190.031534945
cond: 190.031494566
cond: 190.031415275
cond: 190.031399314
cond: 190.031516531
cond: 190.03162681
cond: 190.031578008
cond: 190.031433283
cond: 190.031558435
con

In [314]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [315]:
traj_optimizer.calc_frame_traj()

In [316]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [317]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x_norm)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [318]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.1799,-0.0025,0.0117,0.0032,0.0342,0.0181,-0.1904,-0.0875,-0.1733,-0.2935,-0.4342,-0.5644,2.4547
q2,0.2994,0.1253,-0.0082,-0.0863,0.8471,0.3598,0.3675,-0.0011,-0.2447,-0.0578,-0.4937,0.739,-0.3508
q4,0.3067,0.067,0.0632,0.0824,0.5503,-0.5557,-0.415,0.0015,-0.2342,-0.1393,-0.0762,0.0542,0.0279


---
# Data processing

In [497]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor

## Load data

### Load robot model

In [589]:
# Names as Arguments
# model_name = 'test_psm_long'
# model_name = 'mtm_3links'
# model_name = 'mtm'
# model_name = 'mtm_spring'
model_name = 'mtm_2spring_tendon'
#model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder,model_name)

trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
dof, fourier_order, base_freq, traj_optimizer_result, reg_norm_mat = load_data(trajectory_folder,
                                                                               training_trajectory_name)
print("dof: {}".format(dof))
print("Fourier order: {}".format(fourier_order))
print("Base frequency: {}".format(base_freq))

dof: 7
Fourier order: 6
Base frequency: 0.1


In [590]:
robot_model.coordinates

### Load traning data set

In [591]:
training_trajectory_name = 'one'

results_folder = 'data/' + model_name +'/measured_trajectory/'
training_results_data_file = results_folder + training_trajectory_name + '_results.csv'

trajectory_sampling_rate = 500
t_train, q_raw_train, dq_raw_train, tau_raw_train = load_trajectory_data(training_results_data_file,
                                                                   trajectory_sampling_rate)

# Reverse the process to get the original effort for each motor for the MTM
if 'mtm' in model_name:
    tau_raw_train[:,1] = tau_raw_train[:,1] - tau_raw_train[:,2] + 0.5515/0.8235 * tau_raw_train[:, 3]
    tau_raw_train[:,2] = tau_raw_train[:,2] - 0.5515/0.8235 * tau_raw_train[:, 3]

(<type 'numpy.ndarray'>, (14989, 21))


### Load traning data set

In [592]:
test_trajectory_name = 'one'

results_folder = 'data/' + model_name +'/measured_trajectory/'
test_results_data_file = results_folder + test_trajectory_name + '_results.csv'

trajectory_sampling_rate = 500
t_test, q_raw_test, dq_raw_test, tau_raw_test = load_trajectory_data(test_results_data_file,
                                                                trajectory_sampling_rate)

# Reverse the process to get the original effort for each motor for the MTM
if 'mtm' in model_name:
    tau_raw_test[:,1] = tau_raw_test[:,1] - tau_raw_test[:,2] + 0.5515/0.8235 * tau_raw_test[:, 3]
    tau_raw_test[:,2] = tau_raw_test[:,2] - 0.5515/0.8235 * tau_raw_test[:, 3]

(<type 'numpy.ndarray'>, (14989, 21))


## Calculate filter cut-off frequency

In [593]:
# times of the highest frequency in the Fourier series
fc_mult = 4.0

fc = base_freq * fourier_order * fc_mult
print("Cut frequency: {}".format(fc))

Cut frequency: 2.4


## Differentiation and filtering

### Traning data

In [596]:
t_cut_train, q_f_train, dq_f_train, ddq_f_train, tau_f_train, q_raw_cut_train, tau_raw_cut_train =\
    diff_and_filt_data(dof, 1.0/trajectory_sampling_rate,
                       t_train, q_raw_train, dq_raw_train, tau_raw_train, fc, fc, fc, fc)
plot_trajectory_data(t_cut_train, q_raw_cut_train, q_f_train, dq_f_train, ddq_f_train,
                     tau_raw_cut_train, tau_f_train)

q_raw shape: (14989, 7)
plot shape: 470


### Test data

In [597]:
t_cut_test, q_f_test, dq_f_test, ddq_f_test, tau_f_test, q_raw_cut_test, tau_raw_cut_test =\
    diff_and_filt_data(dof, 1.0/trajectory_sampling_rate,
                       t_test, q_raw_test, dq_raw_test, tau_raw_test, fc, fc, fc, fc)
plot_trajectory_data(t_cut_test, q_raw_cut_test, q_f_test, dq_f_test, ddq_f_test,
                     tau_raw_cut_test, tau_f_test)

q_raw shape: (14989, 7)
plot shape: 470


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [598]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b_train, tau_s_train = gen_regressor(base_param_num, H_b_func, q_f_train,
                                       dq_f_train, ddq_f_train, tau_f_train)

In [599]:
print(W_b_train.shape, tau_s_train.shape)
#print np.linalg.cond(W_b_train)

((102123, 71), (102123,))


In [600]:
xb_ols = np.linalg.lstsq(W_b_train, tau_s_train)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [601]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*L7xy,-6.90759198035159e-06
1.0*L7yz,-3.6786037081677e-06
1.0*L7xz,-1.3357032513696682e-05
1.0*L6xy,0.0004521252027048753
1.0*L6yz,-4.432613934019446e-05
1.0*L6xz,-0.0002171893832076605
1.0*L6xx - 1.0*L6yy + 1.0*L7xx,0.002232447619316417
1.0*L5xy,-0.0007454825721651118
1.0*L7zz,2.938984649962853e-05
1.0*L5xz,-0.0005527423797967281


## Compare measured torque and predicted torque on the training set

### Generate predicted torque

In [602]:
tau_p_train = np.zeros(tau_f_train.shape)
tau_ps_train = W_b_train.dot(xb_ols)
for i in range(dof):
    tau_p_train[:, i] = tau_ps_train[i::dof]
    

### Evaluate regression

In [603]:
var_regression_error_ols = np.linalg.norm(tau_ps_train - tau_s_train) / (tau_ps_train.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.0001735037344059614
standard deviation of xb using OLS:
[1.35895388e-06 5.16041457e-07 5.18048088e-07 1.33135491e-05
 1.42779579e-05 9.42141238e-06 2.90655629e-05 2.20325938e-05
 1.47487399e-06 1.41412864e-05 2.06153772e-06 1.15608940e-05
 2.02829455e-06 3.75298570e-05 2.01120720e-05 3.15274501e-05
 4.13007521e-05 1.63549268e-05 2.90125647e-05 4.04814407e-05
 3.38139963e-05 2.32687977e-05 2.25618809e-05 2.85410170e-05
 2.97052365e-05 6.72416612e-04 2.22299772e-04 1.97408382e-03
 1.24592418e-04 2.20550226e-04 1.00230612e-04 2.81138077e-04
 2.24640256e-04 2.21817097e-04 1.87820013e-04 3.11310607e-05
 4.26012965e-05 1.15128860e-04 7.19843499e-05 3.42605543e-04
 2.10944247e-04 1.03212192e-04 7.83480937e-05 5.41396162e-04
 6.93432718e-04 4.70149665e-04 4.65427212e-04 5.18469801e-05
 9.12569580e-05 2.16880299e-05 2.47587573e-04 2.29038024e-04
 3.69159169e-04 2.77564377e-03 4.53651890e-04 1.40009693e-04
 1.64936222e-04 1.09866121e-04 1.46873941e-04 1.

### Plot measured torque and predicted torque

In [605]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut_train[:plot_data_num], tau_f_train[:plot_data_num, :],
                   tau_p_train[:plot_data_num, :], robot_def.coordinates_joint_type)

## Compare measured torque and predicted torque on the test set

In [607]:
W_b_test, tau_s_test = gen_regressor(base_param_num, H_b_func, q_f_test,
                                       dq_f_test, ddq_f_test, tau_f_test)

tau_p_test = np.zeros(tau_f_test.shape)
tau_ps_test = W_b_test.dot(xb_ols)
for i in range(dof):
    tau_p_test[:, i] = tau_ps_test[i::dof]

plot_data_num = int(1 / base_freq * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut_test[:plot_data_num], tau_f_test[:plot_data_num, :],
                   tau_p_test[:plot_data_num, :], robot_def.coordinates_joint_type)

## Weighted Least Square (WLS)

### Training data set

In [613]:
# weight = np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))
weight = np.max(tau_f_train, axis=0) - np.min(tau_f_train, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([3.64936594, 2.00191696, 2.71843345, 0.46136968, 0.1621324 ,
       0.10858765, 0.00701173])

In [614]:
W_b_train.shape, tau_s_train.shape

In [615]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b_train.shape[0]/weight.shape[0])

In [616]:
weights.shape

In [617]:
W_b_wls_train = np.multiply(W_b_train, np.asmatrix(weights).transpose())
tau_s_wls_train = np.multiply(tau_s_train, weights)

In [618]:
xb_wls = np.linalg.lstsq(W_b_wls_train, tau_s_wls_train)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [619]:
#np.set_printoptions(precision=2)
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                   

In [620]:
tau_p_wls_train = np.zeros(tau_f_train.shape)
tau_ps_wls_train = W_b_train.dot(xb_wls)
for i in range(dof):
    tau_p_wls_train[:, i] = tau_ps_wls_train[i::dof]

In [621]:
plot_meas_pred_tau(t_cut_train[:plot_data_num], tau_f_train[:plot_data_num, :],
                   tau_p_wls_train[:plot_data_num, :], robot_def.coordinates_joint_type)

In [622]:
np.linalg.norm(tau_f_train[:plot_data_num, :] - tau_p_wls_train[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f_train[:plot_data_num, :], axis=0)

array([0.08, 0.28, 0.16, 0.26, 0.29, 0.22, 0.27])

In [623]:
robot_model.bary_param

In [527]:
robot_def.d_coordinates[0] != robot_def.dq_for_frame[1]

False

In [547]:
robot_model.coordinates[1].name[1:]

'2'

### Test data set

In [627]:
weight = np.max(tau_f_test, axis=0) - np.min(tau_f_test, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b_test.shape[0]/weight.shape[0])

W_b_wls_test = np.multiply(W_b_test, np.asmatrix(weights).transpose())
tau_s_wls_test = np.multiply(tau_s_test, weights)

tau_p_wls_test = np.zeros(tau_f_test.shape)
tau_ps_wls_test = W_b_test.dot(xb_wls)
for i in range(dof):
    tau_p_wls_test[:, i] = tau_ps_wls_test[i::dof]

plot_meas_pred_tau(t_cut_test[:plot_data_num], tau_f_test[:plot_data_num, :],
                   tau_p_wls_test[:plot_data_num, :], robot_def.coordinates_joint_type)

In [629]:
np.linalg.norm(tau_f_test[:plot_data_num, :] - tau_p_wls_test[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f_test[:plot_data_num, :], axis=0)

array([0.08, 0.28, 0.16, 0.26, 0.29, 0.22, 0.27])

## Convex optimization

### Generate regressor matrix for barycentric parameters

In [666]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W_train, tau_s_train = gen_regressor(bary_param_num, H_func, q_f_train, dq_f_train, ddq_f_train, tau_f_train)

In [667]:
W_w_train = np.multiply(W_train, np.asmatrix(weights).transpose())
tau_w_s_train = np.multiply(tau_s_train, weights)

In [632]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [638]:
sdp_constraints = []
spring_constraints = []
if model_name == 'mtm' or model_name == 'mtm_2spring_tendon':
    sdp_constraints = [(2, 20, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (1, 10, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (1, 10, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.2, 5, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.1, 2, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.05, 0.5, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.05, 0.5, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (1, 10, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
    spring_constraints = [(250, 300), (0.001, 0.1)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
else:
    raise Exception("No robot name matches " + model_name)

In [640]:


sdp_opt_std = SDPOpt(W_w_train, tau_w_s_train, robot_model, sdp_constraints, spring_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (102123, 122)
Regressand shape: (102123,)
Creating variables...
Creating optimization objective...
Creating constraints...
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 4727149, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 15000, normalize = 1, scale = 1.00
Variables n = 123, constraints m = 102451
Cones:	linear vars: 137
	soc vars: 102125, soc blks: 1
	sd vars: 189, sd blks: 9
Setup time: 1.92e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 3.55e+01  9.94e-01  1.00e+00 -1.04e+04 -1.83e-03  5.05e-12  2.54

  9400| 7.57e-02  9.49e-05  1.33e-03  4.59e-01  4.61e-01  2.64e-12  6.22e+02 
  9500| 7.57e-02  9.40e-05  1.34e-03  4.64e-01  4.66e-01  3.78e-13  6.27e+02 
  9600| 7.56e-02  9.74e-05  1.35e-03  4.69e-01  4.71e-01  3.78e-13  6.33e+02 
  9700| 7.56e-02  1.01e-04  1.36e-03  4.73e-01  4.76e-01  1.13e-12  6.39e+02 
  9800| 7.56e-02  9.90e-05  1.37e-03  4.78e-01  4.81e-01  3.78e-13  6.44e+02 
  9900| 7.56e-02  1.03e-04  1.39e-03  4.83e-01  4.86e-01  3.78e-13  6.50e+02 
 10000| 7.56e-02  1.03e-04  1.40e-03  4.88e-01  4.91e-01  3.78e-13  6.56e+02 
 10100| 7.56e-02  1.01e-04  1.41e-03  4.93e-01  4.96e-01  1.13e-12  6.62e+02 
 10200| 7.56e-02  1.03e-04  1.42e-03  4.98e-01  5.00e-01  3.78e-13  6.67e+02 
 10300| 7.56e-02  1.01e-04  1.43e-03  5.03e-01  5.05e-01  3.78e-13  6.73e+02 
 10400| 7.56e-02  9.91e-05  1.44e-03  5.07e-01  5.10e-01  3.78e-13  6.78e+02 
 10500| 7.56e-02  1.00e-04  1.45e-03  5.12e-01  5.15e-01  1.13e-12  6.83e+02 
 10600| 7.56e-02  9.56e-05  1.46e-03  5.17e-01  5.20e-01  3.78e-

## Compare measured torque and predicted torque on training set

### Generate predicted torque

In [669]:
tau_p_sdp_train = np.zeros(tau_f_train.shape)
tau_ps_sdp_train = W_train.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_p_sdp_train[:, i] = tau_ps_sdp_train[i::dof]

In [684]:
np.linalg.norm(tau_f_train[:plot_data_num, :] - tau_p_sdp_train[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f_train[:plot_data_num, :], axis=0)

array([0.08, 0.31, 0.18, 0.26, 0.3 , 0.22, 0.27])

### Plot measured torque and predicted torque

In [670]:
plot_data_num = int(1 / base_freq * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut_train[:plot_data_num] - t_cut_train[0],
                   tau_f_train[:plot_data_num, :],
                   tau_p_sdp_train[:plot_data_num, :],
                   robot_def.coordinates_joint_type)

## Compare measured torque and predicted torque on test set

In [671]:
bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W_test, tau_s_test = gen_regressor(bary_param_num, H_func, q_f_test, dq_f_test, ddq_f_test, tau_f_test)

NameError: name 'tau_sdp_p_test' is not defined

In [673]:
tau_p_sdp_test = np.zeros(tau_f_test.shape)
tau_ps_sdp_test = W_test.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_p_sdp_test[:, i] = tau_ps_sdp_test[i::dof]
plot_data_num = int(1 / base_freq * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut_test[:plot_data_num] - t_cut[0],
                   tau_f_test[:plot_data_num, :],
                   tau_p_sdp_test[:plot_data_num, :],
                   robot_def.coordinates_joint_type)

In [685]:
np.linalg.norm(tau_f_test[:plot_data_num, :] - tau_p_sdp_test[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f_test[:plot_data_num, :], axis=0)

array([0.08, 0.31, 0.18, 0.26, 0.3 , 0.22, 0.27])

In [674]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx    2.59287098939633   ⎤
⎢                           ⎥
⎢L1xy  -4.63330435267689e-14⎥
⎢                           ⎥
⎢L1xz  5.17909685007954e-24 ⎥
⎢                           ⎥
⎢L1yy    2.59287098939632   ⎥
⎢                           ⎥
⎢L1yz  4.60586214662963e-17 ⎥
⎢                           ⎥
⎢L1zz   7.31392188253912e-6 ⎥
⎢                           ⎥
⎢l1x   4.30690070190564e-23 ⎥
⎢                           ⎥
⎢l1y   7.75320298465504e-18 ⎥
⎢                           ⎥
⎢l1z     0.437817674737075  ⎥
⎢                           ⎥
⎢ m₁     3.85448024686883   ⎥
⎢                           ⎥
⎢Fc₁    0.0191747661725284  ⎥
⎢                           ⎥
⎢Fv₁    0.0645796077091047  ⎥
⎢                           ⎥
⎢Fo₁    -0.0629530517963016 ⎥
⎢                           ⎥
⎢L2xx   0.0231657465400414  ⎥
⎢                           ⎥
⎢L2xy   0.0073300746276269  ⎥
⎢                           ⎥
⎢L2xz   0.00235684484784291 ⎥
⎢                           ⎥
⎢L2yy   0.0134474981688839  ⎥
⎢         

In [675]:
from identification import barycentric2standard_params

In [676]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [677]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx    2.54314072642043   ⎤
⎢                           ⎥
⎢I1xy  -4.63330435267689e-14⎥
⎢                           ⎥
⎢I1xz  1.00711630428265e-23 ⎥
⎢                           ⎥
⎢I1yy    2.54314072642042   ⎥
⎢                           ⎥
⎢I1yz  4.69392821741216e-17 ⎥
⎢                           ⎥
⎢I1zz   7.31392188253912e-6 ⎥
⎢                           ⎥
⎢r1x   1.11737521690618e-23 ⎥
⎢                           ⎥
⎢r1y   2.01147819889681e-18 ⎥
⎢                           ⎥
⎢r1z     0.113586695662206  ⎥
⎢                           ⎥
⎢ m₁     3.85448024686883   ⎥
⎢                           ⎥
⎢Fc₁    0.0191747661725284  ⎥
⎢                           ⎥
⎢Fv₁    0.0645796077091047  ⎥
⎢                           ⎥
⎢Fo₁    -0.0629530517963016 ⎥
⎢                           ⎥
⎢I2xx   0.0223251171561595  ⎥
⎢                           ⎥
⎢I2xy   0.00532703958857493 ⎥
⎢                           ⎥
⎢I2xz  -5.13659051766726e-5 ⎥
⎢                           ⎥
⎢I2yy   0.00127885235625489 ⎥
⎢         

In [678]:
sdp_constraints